Classify public transport schedule time slots into demand levels

In [9]:
import pandas as pd
import numpy as np

# Simulated Dataset
data = {
    "time_slot": ["6AM-7AM", "7AM-8AM", "8AM-9AM", "9AM-10AM", "10AM-11AM"],
    "weekday": [1, 1, 1, 0, 0],  # 1 for weekday, 0 for weekend
    "weather": [0, 1, 0, 1, 0],  # 0 for clear, 1 for rainy
    "passenger_count": [20, 50, 80, 30, 25],
    "demand_level": ["Low", "High", "High", "Medium", "Low"]  # Target variable
}

df = pd.DataFrame(data)

# Encode categorical target variable
df["demand_level"] = df["demand_level"].map({"Low": 0, "Medium": 1, "High": 2})

# Prepare features (X) and target (y)
X = df[["weekday", "weather", "passenger_count"]].values
y = df["demand_level"].values




In [10]:
def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    X_scaled = (X - mean) / std
    return X_scaled, mean, std

X_scaled, X_mean, X_std = standardize(X)

# Define the softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Define the logistic regression training function
def logistic_regression(X, y, num_classes, learning_rate=0.01, epochs=1000):
    m, n = X.shape
    weights = np.zeros((n, num_classes))  # Initialize weights
    bias = np.zeros(num_classes)  # Initialize bias
    
    # One-hot encode target labels
    y_one_hot = np.zeros((m, num_classes))
    y_one_hot[np.arange(m), y] = 1
    
    # Gradient descent loop
    for _ in range(epochs):
        z = np.dot(X, weights) + bias
        y_pred = softmax(z)
        
        dw = (1/m) * np.dot(X.T, (y_pred - y_one_hot))
        db = (1/m) * np.sum(y_pred - y_one_hot, axis=0)
        
        weights -= learning_rate * dw
        bias -= learning_rate * db
    
    return weights, bias

In [11]:

# Train the model
num_classes = len(np.unique(y))
weights, bias = logistic_regression(X_scaled, y, num_classes)

In [14]:
import pickle
# Save the model and scaler
with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump((weights, bias, X_mean, X_std), f)
print("Model and scaler saved successfully.")
# Load the model and scaler
with open("logistic_regression_model.pkl", "rb") as f:
    weights, bias, X_mean, X_std = pickle.load(f)

print("Model and scaler loaded successfully.")
    

Model and scaler saved successfully.
Model and scaler loaded successfully.


In [15]:

# Function to make predictions
def predict(X, weights, bias):
    X_scaled = (X - X_mean) / X_std  # Standardize input
    z = np.dot(X_scaled, weights) + bias
    y_pred = softmax(z)
    return np.argmax(y_pred, axis=1)

In [17]:
# Get user input
weekday = int(input("Is it a weekday? (1 for Yes, 0 for No): "))
weather = int(input("What is the weather? (1 for Rainy, 0 for Clear): "))
passenger_count = int(input("Enter the number of passengers: "))

# Prepare the input for prediction
X_input = np.array([[weekday, weather, passenger_count]])

# Make prediction
predicted_demand = predict(X_input, weights, bias)

# Map prediction to demand level
demand_map = {0: "Low", 1: "Medium", 2: "High"}
print(f"Predicted Demand Level: {demand_map[predicted_demand[0]]}")

Predicted Demand Level: Low
